## Reed-Solomon/Reed-Muller Concatenated Code

In [88]:
import numpy as np
import itertools

class RSRMCode:
    
    def __init__(self, n_RS, k_RS, q_RS, r_RM, m_RM, alpha = None):
        
        if not (k_RS < n_RS and n_RS <= q_RS):
            raise ValueError('Invalid RS values for n, k, and q.')
            
        self.p0, self.k_RM = is_prime_power(q_RS, get_data = True)
            
        self.n_RS = n_RS
        self.k_RS = k_RS
        self.q_RS = q_RS
        self.r_RM = r_RM
        self.m_RM = m_RM
        
        self.n_RM = 2**self.m_RM
        k_RM = sum(binomial(self.m_RM,i) for i in range(self.r_RM + 1))
        if k_RM != self.k_RM:
            raise ValueError('Wrong parameters, the RM dimension does not match the power of the extension field')
        self.d_RM = 2**(self.m_RM - self.r_RM)
        
        self.d_RS = self.n_RS - self.k_RS + 1
        self.tau_RS = floor((self.n_RS-self.k_RS)/2)
        
        # Initializing field
        self.BF = GF(self.p0) # base field
        self.EF = GF(self.q_RS) # extension field
        self.R = PolynomialRing(self.EF, 'X')
        self.p = self.EF.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n_RS)])
        else:
            self.alpha = alpha
        
        # Constructing RS generator matrix
        self.G_RS = matrix(self.EF, self.k_RS, self.n_RS, lambda i,j : self.alpha[j]**i)
        
        # Constructing RM generator matrix
        self.G_RM = matrix(self.BF, self.k_RM, self.n_RM, self.generateG(self.r_RM, self.m_RM))
        
    def generateG(self, r, m):
        ones = np.repeat(1, 2**m)
        bases = [np.array([(i // 2**a) % 2 for i in range(2**m)]) for a in range(m)]
        if r == 1:
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases)])
            return G
        elif r > 1:
            others = []
            for i in range(2,m-1):
                others.extend([math.prod(x) for x in itertools.combinations(bases, i)])
            
            G = np.concatenate([ones.reshape(1,2**m), np.stack(bases), np.stack(others)])
            return G
        else:
            raise ValueError('wrong value of r')
        
        
    def Encoding(self, m, zeropad = True):
        
        m = self._IntToPol(m)
        
        rem = len(m) % self.k_RS
        
        if rem != 0:
            if zeropad:
                m.extend([self.EF(0)]*(self.k_RS-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c_RS = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k_RS):
            c_RS.extend(self.EncodeChunkRS(m[i:i+self.k_RS]))
        
        
        c_RS = self._PolToInt(c_RS)
        c_RS = self._IntToByteString(c_RS)
        
        c_RSRM = []
        
        # Encoding each chunk of size k
        for i in range(0, len(c_RS), self.k_RM):
            c_RSRM.extend(self.EncodeChunkRM(c_RS[i:i+self.k_RM]))
        
        return vector(self.BF, c_RSRM)
        
        
    def EncodeChunkRS(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RS:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.EF, chunk) * self.G_RS
        return c
    
    def EncodeChunkRM(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k_RM:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.BF, chunk) * self.G_RM

        return c
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n_RM != 0:
            raise ValueError('Invalid input size')
        
        c_RM = []
        
        for i in range(0,len(r),self.n_RM):
            c_RM.extend(self.DecodeChunkRM(r[i:i+self.n_RM]))
            
        c_RM = self._ByteStringToInt(c_RM)
        c_RM = self._IntToPol(c_RM)
        
        c_RSRM = []
        
        for i in range(0,len(c_RM),self.n_RS):
            c_RSRM.extend(self.DecodeChunkRS(c_RM[i:i+self.n_RS]))
            
        c_RSRM = self._PolToInt(c_RSRM)
        return c_RSRM
    
    
    def DecodeChunkRS(self, chunk):
        # Bivariate Interpolation algorithm
        
        if len(chunk) != self.n_RS:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.EF, self.n_RS, self.tau_RS + self.k_RS, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.EF, self.n_RS, self.tau_RS + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau_RS+self.k_RS]))
        Q1 = self.R(list(sol[self.tau_RS+self.k_RS:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.EF(0)]*(self.k_RS-len(out)))

        return out
    
    
    def DecodeChunkRM(self, word):
        # Reed Decoding algorithm
        
        if (len(word) != self.n_RM):
            raise ValueError('Invalid chunk size')
        
        decoded = []
        
        if r == 2:
            variables = [i for i in range(1, self.m_RM + 1)]
            combinations = []
            combinations.append(list(itertools.combinations(variables,2)))
            combinations = list(combinations)
            
            for i in range(factorial(self.m_RM)-1, -1, -1):
                
                # compute the complementary set T
                T = [i for i in range(1,self.m_RM+1)]
                for i in range(self.r_RM):
                    T.remove(combinations[0][i])
                    
                # compute the current set S
                S = [1] * self.m_RM
                for i in range(len(T)):
                    S[T[i] - 1] = 0
                S = vector(self.BF, S)
                
                # compute all combinations of the variables in the complementary set T
                combinations = []
                for i in range(len(T)+1):
                    combinations.append(list(itertools.combinations(T,i)))
                combinations = list(combinations)
                    
            
        
        # first decode a_1, ..., a_m, in reverse order (a_m, ..., a_1) 
        for i in range(self.m_RM,0,-1):
            
            # compute the complementary set T 
            T = [i for i in range(1,self.m_RM+1)]
            del T[i-1]    
            
            # compute the current set S
            S = [1] * self.m_RM
            for i in range(len(T)):
                S[T[i] - 1] = 0
            S = vector(self.BF, S)

            # compute all combinations of the variables in the complementary set T
            combinations = []
            for i in range(len(T)+1):
                combinations.append(list(itertools.combinations(T,i)))
            combinations = list(combinations)
            
            # compute all the votings
            votes = []
            for i in range(len(combinations)):
                for j in range(len(combinations[i])):
                    voting_set = [0] * self.m_RM
                    for k in range(len(combinations[i][j])):
                        voting_set[combinations[i][j][k] - 1] = 1
                    voting_set = vector(self.BF, voting_set)
                    votes.append(word[ZZ(voting_set.list(),2)] + word[ZZ((voting_set+S).list(),2)])
            
            # decode coefficient by comparing number of ones with number of zeros in the list of votings
            decoded = self.MajorityDecoding(votes) + decoded
        
        # decode a_0 by subtracting the part with a_1, ..., a_m
        self.G_RM_part = matrix(self.BF, self.k_RM - 1, self.n_RM, self.G_RM[1:][:])
        word_part = vector(self.BF, decoded) * self.G_RM_part
        decoded = self.MajorityDecoding((word+word_part).list()) + decoded
        
        return decoded
    
            
    def MajorityDecoding(self, word):
        # Decode by comparing the number of ones with the number of zeros
        
        if word.count(1) >= floor(len(word) / 2):
            return [1]
        elif word.count(1) < floor(len(word) / 2):
            return [0]
        else:
            return "decoding failure"
    
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.EF:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)

    def _IntToByteString(self, int_array):

        if(self.q_RS != 2**self.k_RM):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of integers less than 256 to binary representation

        if any([(item > 2**self.k_RM - 1 or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return list(''.join([format(item, '08b') for item in int_array]))
    
    def _ByteStringToInt(self, byte_string):

        if(self.q_RS != 2**self.k_RM):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of 8 bit binary representations to integers

        #if (len(byte_string) % self.k_RM != 0) or any([not (bit == '1' or bit == '0') for bit in byte_string]):
        #    raise ValueError('Invalid byte string')

        m_out = []
        for i in range(0,len(byte_string), self.k_RM):
            current = byte_string[i:i+self.k_RM]
            m_out.append(int("".join(str(x) for x in current), 2))
            
        return m_out
        
    
    def _IntToPol(self, m):
        # Converts array of integers less than q to elements of Field.

        m_out = []

        for s in m:
            if not s < self.q_RS:
                raise ValueError('Invalid symbol')
            m_out.append(self.EF(ZZ(s).digits(self.p0)))

        return(m_out)

In [89]:
C = RSRMCode(15,7,2**8,1,7)

In [90]:
message = [1,2,3,4,5,6,7]
c = C.Encoding(message)
c

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 

In [91]:
#e = vector(GF(2), [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
#r = c + e
#r

In [92]:
d = C.Decoding(c)
print(d)

[1, 2, 3, 4, 5, 6, 7]


In [47]:
sum(binomial(3,i) for i in range(1 + 1))

4

In [20]:
type(i)

<class 'sage.rings.number_field.number_field_element_quadratic.NumberFieldElement_gaussian'>